In [3]:
import json
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
users_df = pd.read_csv('/content/drive/MyDrive/targilimP/bipper_users.csv')

In [6]:
with open('/content/drive/MyDrive/targilimP/bipper_activity_details.json') as f:
    social_data = json.load(f)
records = []
for entry in social_data:
    for message in entry['messages']:
        records.append({
            'user_id': entry['user_id'],
            'to_user': message['to_user'],
            'timestamp': message['timestamp'],
            'friends_count': len(entry['friends']),
            'message': message['content']
        })

df_messages = pd.DataFrame(records)
df_messages.head()

,user_id,to_user,timestamp,friends_count,message
0,1,4,2024-08-24T21:53:57.083653,3,"Hey David, how are you?"
1,2,13,2024-04-27T21:53:57.091285,1,"Hey Mark, how are you?"
2,2,13,2024-09-25T21:53:57.092404,1,"Let's meet at Central Park on 2023-11-13, Mark!"
3,2,13,2024-02-13T21:53:57.093354,1,"Hey Mark, how are you?"
4,2,13,2024-01-31T21:53:57.094258,1,"Let's meet at Riverwalk Cafe on 2024-01-06, Mark!"


In [7]:
merged_df = pd.merge(df_messages, users_df, on='user_id', how='inner')#  פעולה זו מאחדת לנו בין הטבלאות בצורה של השוואה של המפתחות
print(merged_df.head())


   user_id  to_user                   timestamp  friends_count  \
0        1        4  2024-08-24T21:53:57.083653              3   
1        2       13  2024-04-27T21:53:57.091285              1   
2        2       13  2024-09-25T21:53:57.092404              1   
3        2       13  2024-02-13T21:53:57.093354              1   
4        2       13  2024-01-31T21:53:57.094258              1   

                                             message      name  age gender  \
0                            Hey David, how are you?      Ian1   51      M   
1                             Hey Mark, how are you?  Charlie2   51      F   
2    Let's meet at Central Park on 2023-11-13, Mark!  Charlie2   51      F   
3                             Hey Mark, how are you?  Charlie2   51      F   
4  Let's meet at Riverwalk Cafe on 2024-01-06, Mark!  Charlie2   51      F   

      city   join_date  
0  Seattle  2024-03-24  
1  Atlanta  2024-02-14  
2  Atlanta  2024-02-14  
3  Atlanta  2024-02-14  
4  Atlant

In [8]:
rows, columns = merged_df.shape#פעולה שמחזירה כטאפל את מספר השורות ומספר העמודות
print(f'Number of rows: {rows}, Number of columns: {columns}')

Number of rows: 5460, Number of columns: 10


In [9]:
merged_df['join_date'] = pd.to_datetime(merged_df['join_date'])#הטיפוס של העמודה של התאריך היא מסוג אובייקט ולכן נשנהה אותו לההיות מסוג תאריךך
newest = merged_df.loc[merged_df['join_date'].idxmax()]#ניגש לעמודה של ההאריך ואז נפעיל את הפונקציה שתחזיר לנו את הערך הגדול ביותר בעמודה
print(newest.user_id)

465


In [10]:
target_user_id=2
message_counts = merged_df.groupby('user_id').size()#סופרים עבור כל משתמש בטבלה כמה הודעות הוא שלח
print(f'Number of messages sent by user {target_user_id}: {message_counts.get(target_user_id, 0)}')#מדפיס את כמות ההודעות שמשתמש מספר 2 שלח בצורה שהוא מסנן רק לפי המספר שנתנו לו ואם הוא לא קיים מחזיר 0


Number of messages sent by user 2: 6


In [11]:
meeting_requests = merged_df[merged_df['message'].str.contains("Let's meet", case=False, na=False)]
print(meeting_requests)

      user_id  to_user                   timestamp  friends_count  \
2           2       13  2024-09-25T21:53:57.092404              1   
4           2       13  2024-01-31T21:53:57.094258              1   
5           2       13  2024-05-19T21:53:57.095395              1   
7           3       13  2024-10-08T21:53:57.098070              1   
8           3       13  2024-01-25T21:53:57.099169              1   
...       ...      ...                         ...            ...   
5452     1000        3  2023-12-27T21:54:01.910856              4   
5453     1000       15  2024-10-01T21:54:01.911253              4   
5457     1000       15  2024-04-20T21:54:01.914068              4   
5458     1000        1  2023-12-30T21:54:01.914542              4   
5459     1000       10  2023-11-17T21:54:01.915063              4   

                                                message        name  age  \
2       Let's meet at Central Park on 2023-11-13, Mark!    Charlie2   51   
4     Let's meet at

In [17]:
# עבור סעיף 6 אני מבינה שכדי לדעת מה היחס בין מספר ההודעות למספר החברים אני צריכה לעשות את זה בכמה שלבים
# בשלב הראשון אני אבדוק עבור כל יוזר כמה חברים יש לו
#בשלב השני אני אבדוק כמה הודעות יש לו
#בשלב האחרון אחלק את כמות ההודעות לכמות החברים ואקח משם את הקטן ביותר
# הסתבכתי פה עם הכתיבה של הקוד עצמו אבל זה הרעיון מקווה שזה בסדר

In [19]:
# חישוב מספר המשתמשים השונים שכל משתמש מקיים איתם אינטראקציה
unique_contacts = df_messages.groupby('user_id')['to_user'].nunique().reset_index()

# סינון המשתמשים שמקיימים אינטראקציה עם משתמש אחד בלבד
single_contact_users = unique_contacts[unique_contacts['to_user'] == 1]

# הצגת התוצאה
print(single_contact_users)



     user_id  to_user
0          1        1
1          2        1
2          3        1
7          8        1
15        16        1
..       ...      ...
974      975        1
978      979        1
986      987        1
996      997        1
998      999        1

[343 rows x 2 columns]
